In [ ]:
from pywinauto import application
import time
import os

# 증권사 API 자동 실행
os.system('taskkill /IM coStarter* /F /T')
os.system('taskkill /IM CpStart* /F /T')
os.system('wmic process where "name like \'%coStarter%\'" call terminate')
os.system('wmic process where "name like \'%CpStart%\'" call terminate')
time.sleep(5)        

app = application.Application()
app.start('C:\CREON\STARTER\coStarter.exe /prj:cp /id:아이디 /pwd:비밀번호 /pwdcert:인증서 비밀번호 /autostart')
time.sleep(60)

In [ ]:
import win32com.client
import ctypes

# 서버 접속 확인
objCpCybos = win32com.client.Dispatch("CpUtil.CpCybos")
bConnect = objCpCybos.IsConnect

def InitPlusCheck():
    # 프로세스가 관리자 권한으로 실행 여부
    if ctypes.windll.shell32.IsUserAnAdmin():
        print('정상: 관리자권한으로 실행된 프로세스입니다.')
    else:
        print('오류: 일반권한으로 실행됨. 관리자 권한으로 실행해 주세요')
        return False

    if (bConnect == 1): #접속 - 1, 비접속 - 0
        print("정상: 서버에 접속되었습니다.")
    else:
        print("오류: 서버에 연결되지 않았습니다. ")
        return False

InitPlusCheck()

정상: 관리자권한으로 실행된 프로세스입니다.
정상: 서버에 접속되었습니다.


In [ ]:
import win32com.client
import pandas as pd
from datetime import datetime
import time

# 오브젝트 가져오기
g_objCodeMgr = win32com.client.Dispatch('CpUtil.CpCodeMgr')
g_objCpStatus = win32com.client.Dispatch('CpUtil.CpCybos')
g_objCpTrade = win32com.client.Dispatch('CpTrade.CpTdUtil')

objRq = win32com.client.Dispatch("CpSysDib.CpSvr7210d")

objRq.SetInputValue(0, '0')
# 0 전체 1 거래소 2 코스닥 3 업종 4 관심종목
objRq.SetInputValue(1, ord('0'))
# 0 수량 1 금액
objRq.SetInputValue(2, 0)
# 0 종목 1 외국인 2 기관계 3 보험기타 4 투신..
objRq.SetInputValue(3, ord('0'))
# 0 상위순 1 하위순

sumcnt = 0
df = None
df = pd.DataFrame(columns=('code', '종목명', '현재가', '대비', '대비율', '거래량', '외국인', '기관계', '보험기타금융', '투신', '은행', '연기금', '사모펀드', '기타법인'))

while True:

    remainCount = g_objCpStatus.GetLimitRemainCount(1)  # 1 시세 제한

    if remainCount <= 0:
        print('시세 연속 조회 제한 회피를 위해 sleep', g_objCpStatus.LimitRequestRemainTime)
        time.sleep(g_objCpStatus.LimitRequestRemainTime / 1000)

    objRq.BlockRequest()
 
    cnt = objRq.GetHeaderValue(0)
    sumcnt += cnt
    #date = objRq.GetHeaderValue(1) # 집계날짜
    #time = objRq.GetHeaderValue(2)  # 집계시간
    print(cnt)
 
    for i in range(cnt):
        item = {}
        item['code'] = objRq.GetDataValue(0, i)
        item['종목명'] = objRq.GetDataValue(1, i)
        item['현재가'] = objRq.GetDataValue(2, i)
        item['대비'] = objRq.GetDataValue(3, i)
        item['대비율'] = objRq.GetDataValue(4, i)
        item['거래량'] = objRq.GetDataValue(5, i)
        item['외국인'] = objRq.GetDataValue(6, i)
        item['기관계'] = objRq.GetDataValue(7, i)
        item['보험기타금융'] = objRq.GetDataValue(8, i)
        item['투신'] = objRq.GetDataValue(9, i)
        item['은행'] = objRq.GetDataValue(10, i)
        item['연기금'] = objRq.GetDataValue(11, i)
        item['국가지자체'] = objRq.GetDataValue(12, i)
        item['기타법인'] = objRq.GetDataValue(13, i)

        df.loc[len(df)] = item

        # 1000개 처리
    if sumcnt > 3000:
        break
    # 연속 처리
    if objRq.Continue != True:
        break

600
600
600
600
600
201


In [ ]:
df

,code,종목명,현재가,대비,대비율,거래량,외국인,기관계,보험기타금융,투신,은행,연기금,사모펀드,기타법인
0,A000020,동화약품,15500,-200,-1.27,65553,-176,0,0,0,0,0,NaN,0
1,A000040,KR모터스,936,-10,-1.06,265104,789,0,0,0,0,0,NaN,0
2,A000050,경방,12850,-200,-1.53,4499,-47,0,0,0,0,0,NaN,0
3,A000060,메리츠화재,14750,-100,-0.67,366610,-188662,17000,-4000,21000,0,0,NaN,-1000
4,A000070,삼양홀딩스,88400,-700,-0.79,14541,-2,0,0,0,0,0,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3196,Q590017,미래에셋 미국 고배당주 ETN(H),15080,0,0.00,0,0,0,0,0,0,0,NaN,0
3197,Q590018,미래에셋 중국 심천 100 ETN,20690,-235,-1.12,81,0,0,0,0,0,0,NaN,0
3198,Q700001,하나 코스피 변동성추세 추종 양매도 ETN,7220,0,0.00,0,0,0,0,0,0,0,NaN,0
3199,Q700002,하나 KRX300 ETN,11300,0,0.00,0,0,0,0,0,0,0,NaN,0


In [ ]:
df.to_excel('종목별 투자자 현황.xlsx', encoding='cp949')

In [ ]:
import win32com.client
import pandas as pd
from datetime import datetime
import time

# 오브젝트 가져오기
g_objCodeMgr = win32com.client.Dispatch('CpUtil.CpCodeMgr')
g_objCpStatus = win32com.client.Dispatch('CpUtil.CpCybos')
g_objCpTrade = win32com.client.Dispatch('CpTrade.CpTdUtil')

objRq = win32com.client.Dispatch("CpSysDib.MarketEye")

codeList = g_objCodeMgr.GetStockListByMarket(1)  # 거래소
codeList2 = g_objCodeMgr.GetStockListByMarket(2)  # 코스닥
allcodelist = codeList + codeList2

rqField = [0, 20] #요청 필드
rqCodeList = []



sumcnt = 0
df = None
df = pd.DataFrame(columns=('code', '종목명', '상장주식수'))

for i, code in enumerate(allcodelist):
    rqCodeList.append(code)
    if len(rqCodeList) == 200:
        print(len(rqCodeList))
        rqCodeList = []

while True:
    remainCount = g_objCpStatus.GetLimitRemainCount(1)  # 1 시세 제한
    if remainCount <= 0:
        print('시세 연속 조회 제한 회피를 위해 sleep', g_objCpStatus.LimitRequestRemainTime)
        time.sleep(g_objCpStatus.LimitRequestRemainTime / 1000)
 
    objRq.SetInputValue(0, rqField)
    # 0 기본, 0 종목코드 20 총주식수
    objRq.SetInputValue(1, rqCodeList)
    # 1 종목코드 또는 종목코드 배열(최대 200종목까지 가능)

    objRq.BlockRequest()

    cnt = objRq.GetHeaderValue(2)
    sumcnt += cnt
    print(cnt)
    print(sumcnt)
    
    for i in range(cnt):
        item = {}
        item['code'] = objRq.GetDataValue(0, i)
        item['상장주식수'] = objRq.GetDataValue(1, i)

        df.loc[len(df)] = item

    if sumcnt > 3000:
        break

200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
59
59
59
118
59
177
59
236
59
295
59
354
59
413
59
472
59
531
59
590
59
649
59
708
59
767
59
826
59
885
59
944
59
1003
59
1062
59
1121
59
1180
59
1239
59
1298
59
1357
59
1416
59
1475
59
1534
59
1593
59
1652
59
1711
59
1770
59
1829
59
1888
59
1947
59
2006
59
2065
59
2124
59
2183
59
2242
59
2301
59
2360
59
2419
59
2478
59
2537
59
2596
59
2655
59
2714
59
2773
59
2832
59
2891
59
2950
59
3009


In [ ]:
import win32com.client
import pandas as pd
from datetime import datetime
import time

# 오브젝트 가져오기
g_objCodeMgr = win32com.client.Dispatch('CpUtil.CpCodeMgr')
g_objCpStatus = win32com.client.Dispatch('CpUtil.CpCybos')
g_objCpTrade = win32com.client.Dispatch('CpTrade.CpTdUtil')

class CpMarketEye:
    def __init__(self):
        self.objRq = win32com.client.Dispatch("CpSysDib.MarketEye")
        self.RpFiledIndex = 0
 
 
    def Request(self, codes, dataInfo):
        # 0: 종목코드 4: 현재가 20: 상장주식수
        rqField = [0, 4, 20]  # 요청 필드
 
        self.objRq.SetInputValue(0, rqField)  # 요청 필드
        self.objRq.SetInputValue(1, codes)  # 종목코드 or 종목코드 리스트
        self.objRq.BlockRequest()
 
        cnt = self.objRq.GetHeaderValue(2)
 
        for i in range(cnt):
            code = self.objRq.GetDataValue(0, i)  # 코드
            cur = self.objRq.GetDataValue(1, i)  # 종가
            listedStock = self.objRq.GetDataValue(2, i)  # 상장주식수
 
            maketAmt = listedStock * cur
            if g_objCodeMgr.IsBigListingStock(code) :
                maketAmt *= 1000
            # print(code, maketAmt)
 
            # key(종목코드) = tuple(상장주식수, 시가총액)
            dataInfo[code] = (listedStock, maketAmt)
 
        return True
 
class CMarketTotal():
    def __init__(self):
        self.dataInfo = {}
 
    def GetAllMarketTotal(self):
        codeList = g_objCodeMgr.GetStockListByMarket(1)  # 거래소
        codeList2 = g_objCodeMgr.GetStockListByMarket(2)  # 코스닥
        allcodelist = codeList + codeList2
 
        objMarket = CpMarketEye()
        rqCodeList = []
        for i, code in enumerate(allcodelist):
            rqCodeList.append(code)
            if len(rqCodeList) == 200:
                objMarket.Request(rqCodeList, self.dataInfo)
                rqCodeList = []
                continue
        # end of for
 
        if len(rqCodeList) > 0:
            objMarket.Request(rqCodeList, self.dataInfo)
 
    def PrintMarketTotal(self):
 
        # 시가총액 순으로 소팅
        data2 = sorted(self.dataInfo.items(), key=lambda x: x[1][1], reverse=True)
        print('전종목 시가총액 순 조회 (%d 종목)' % (len(data2)))

        for item in data2:
            name = g_objCodeMgr.CodeToName(item[0])
            listed = item[1][0]
            markettot = item[1][1]
            print(data2)
 
 
if __name__ == "__main__":
    objMarketTotal = CMarketTotal()
    objMarketTotal.GetAllMarketTotal()
    objMarketTotal.PrintMarketTotal()


(27931470, 427351491000)
(96137368, 90465263288)
(27415270, 357769273500)
(120625000, 1755093750000)
(8564271, 757937983500)
(304058, 17148871200)
(70133611, 2289862399150)
(1130138, 22998308300)
(69972959, 4583228814500)
(1180940, 75580160000)
(22812344, 4300126844000)
(23206765, 346941136750)
(470810, 6920907000)
(16523835, 859239420000)
(3996462, 159858480000)
(893038, 54921837000)
(69751600, 159382406000)
(15441262, 1317139648600)
(1686115, 78067124500)
(8035247, 102851161600)
(1133465, 12694808000)
(164080, 4651668000)
(11540400, 192147660000)
(93020173, 1967376658950)
(405363347, 39766144340700)
(118955127, 94569325965)
(13291151, 144208988350)
(185250, 3112200000)
(116738915, 398079700150)
(26814737, 332502738800)
(310336320, 546191923200)
(62000000, 292020000000)
(4000000, 315200000000)
(23728210, 188639269500)
(4160347, 90487547250)
(6694896, 132893685600)
(64242645, 212643154950)
(768000, 5905920000)
(153600, 3041280000)
(1154482, 66959956000)
(6155346, 680165733000)
(1429220